In [100]:
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
import plotly.graph_objects as go
from scipy.stats import multivariate_normal as mvn
import pandas as pd
##### Inputs
# Number of simulations
num_sims = 500

# Risk-free rate
r = 0.02
# Expected returns
means = np.array([0.06, 0.065, 0.08])
# Standard deviations
sds = np.array([0.15, 0.165, 0.21])

# Correlations
corr12 = 0.75
corr13 = 0.75
corr23 = 0.75
# Covariance matrix
C  = np.identity(3)
C[0, 1] = C[1, 0] = corr12
C[0, 2] = C[2, 0] = corr13
C[1, 2] = C[2, 1] = corr23
cov = np.diag(sds) @ C @ np.diag(sds)

# Window length (and initial period)
window = 30

# Length of out-of-sample period
T = 50

n = len(means)
raver = 2


In [101]:
def tangency(means, cov, rf, short_lb):
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''

    n = len(means)
    def f(w):
        mn = w @ means
        sd = np.sqrt(w.T @ cov @ w)
        return -(mn - rf) / sd
    # Initial guess (equal-weighted)
    w0 = (1/n)*np.ones(n)
    # Constraint: fully-invested portfolio
    A = np.ones(n)
    b = 1
    cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
    bnds = [(short_lb, None) for i in range(n)] 
    # Optimization
    wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
    return wgts_tangency

wgts_true = tangency(means,cov,r,short_lb = -3.0)
cal_wgt_true = (wgts_true @ means - r) / (raver * (wgts_true @ cov @ wgts_true))
print(cal_wgt_true)

0.9236475718970099


In [40]:

def gmv(cov, short_lb): 
    '''
    short_lb: lower bound on position weights
    examples: 0  = no short-selling
              -1 = no more than -100% in a given asset
              None=no restrictions on short-selling
    '''    
    n = len(cov)
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    if short_lb==None:
        # No position limits
        G = matrix(np.zeros((n,n)), tc="d")
        h = matrix(np.zeros(n), (n, 1), tc="d")
    else:
        # Constraint: short-sales not allowed
        G = matrix(-np.identity(n), tc="d")
        h = matrix(-short_lb * np.ones(n), (n, 1), tc="d")
    # Fully-invested constraint
    A = matrix(np.ones(n), (1, n), tc="d")
    b = matrix([1], (1, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b, options={'show_progress': False})
    wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
    return wgts_gmv

wgts_gmv = gmv(cov,short_lb=None)


In [95]:

# Simulate data
def simulation(means, cov, short_lb, seed):
	rets = mvn.rvs(means, cov, size=60+T, random_state = seed)
	df = pd.DataFrame(data=rets, columns=['r0','r1','r2'])
	df.columns
	df['mn0']=df['r0'].rolling(60).mean()
	df['mn1']=df['r1'].rolling(60).mean()
	df['mn2']=df['r2'].rolling(60).mean()
	df['sd0']=df['r0'].rolling(60).std()
	df['sd1']=df['r1'].rolling(60).std()
	df['sd2']=df['r2'].rolling(60).std()

	corrs = df[['r0','r1','r2']].rolling(60, min_periods=60).corr()
	df['c01']=corrs.loc[(slice(None),'r0'),'r1'].values
	df['c02']=corrs.loc[(slice(None),'r0'),'r2'].values
	df['c12']=corrs.loc[(slice(None),'r1'),'r2'].values

	model_list = ['true', 'ew', 'est_all', 'est_cov', 'est_sds']
	for model in model_list:
		df['portret_'+model] = np.nan  # portret is the realized portfolio return of the 100% risky asset portfolio
		if model not in ['true','ew']:
			df['wgt0_'+model] = np.nan
			df['wgt1_'+model] = np.nan
			df['wgt2_'+model] = np.nan
		df['wgt_cal_'+model] =np.nan
		df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

	for i in np.arange(60,60+T):
		# Full estimation inputs at each point in time
		means = df[['mn0','mn1','mn2']].iloc[i-1].values
		sds   = df[['sd0','sd1','sd2']].iloc[i-1].values
		corr01 = df.loc[i-1,'c01']
		corr02 = df.loc[i-1,'c02']
		corr12 = df.loc[i-1,'c12']
		C  = np.identity(3)
		C[0, 1] = C[1, 0] = corr01
		C[0, 2] = C[2, 0] = corr02
		C[1, 2] = C[2, 1] = corr12
		cov = np.diag(sds) @ C @ np.diag(sds)

		##### Note: all portfolio weights considered to be beginning of period weights
		##### (so multiply by contemporaneous realized returns)
		# Theoretical optimal weights
		model = 'true'
		df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_true
		df.loc[i,'raver_portret_'+model] = r + cal_wgt_true * (df.loc[i,'portret_'+model] -r)

		# Full estimation tangency portfolio
		model = 'est_all'
		w0, w1, w2 = tangency(means,cov,r,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)

		# Estimate only covariance matrix
		model = 'est_cov'
		w0, w1, w2 = gmv(cov,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)


		# Estimate only standard deviations in covariance matrix
		model = 'est_sds'
		cov[0, 1] = cov[1, 0] = 0.0
		cov[0, 2] = cov[2, 0] = 0.0
		cov[1, 2] = cov[2, 1] = 0.0
		w0, w1, w2 = gmv(cov,short_lb)
		df.loc[i,'wgt0_' + model] = w0
		df.loc[i,'wgt1_' + model] = w1
		df.loc[i,'wgt2_' + model] = w2
		# df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
		wgts = np.array([w0, w1, w2])
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))		
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    

		# Equal-weighted portfolio
		model = 'ew'
		cov[0, 0] = cov[1, 1] = cov[2, 2] = (sds.mean())**2
		wgts = (1/n)*np.ones(n)
		# df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_ew
		df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
		df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
		df.loc[i,'wgt_cal_'+model] = max(0,df.loc[i,'wgt_cal_'+model])
		df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   

	portret_list = ['raver_portret_' +  model for model in model_list]
	collist = ['wgt_cal_'+m for m in ['ew', 'est_all', 'est_cov', 'est_sds']]
	stats = df[portret_list+collist].describe()

	sr_df = pd.DataFrame(dtype=float, columns = ['sr', 'avg_wgt_cal', 'min_wgt_cal', 'mean_raver_port_ret', 'std_raver_port_ret'], index = model_list)
	for model in model_list:
		sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]
	for model in ['ew', 'est_all', 'est_cov', 'est_sds']:
		sr_df.loc[model,'avg_wgt_cal'] = stats.loc['mean','wgt_cal_' +  model]
		sr_df.loc[model,'min_wgt_cal'] = stats.loc['min','wgt_cal_' +  model]
		sr_df.loc[model,'mean_raver_port_ret'] = stats.loc['mean','raver_portret_' +  model]
		sr_df.loc[model,'std_raver_port_ret'] = stats.loc['std','raver_portret_' +  model]
		
	return sr_df, df



In [96]:

sr_df, df = simulation(means, cov, short_lb=None, seed=4)


In [97]:
df

,r0,r1,r2,mn0,mn1,mn2,sd0,sd1,sd2,c01,...,wgt1_est_cov,wgt2_est_cov,wgt_cal_est_cov,raver_portret_est_cov,portret_est_sds,wgt0_est_sds,wgt1_est_sds,wgt2_est_sds,wgt_cal_est_sds,raver_portret_est_sds
0,0.007103,0.131749,0.045605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.145411,0.009785,-0.017832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.186328,0.180547,0.167359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.226433,0.274437,0.269224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.058045,0.073959,-0.016805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.055448,0.206687,0.069130,0.036566,0.038526,0.029006,0.135927,0.164814,0.213440,0.775175,...,0.163361,-0.073878,0.372264,0.042017,0.108089,0.475759,0.330539,0.193701,0.771414,0.087953
106,-0.254843,-0.356827,-0.470513,0.031350,0.030285,0.016239,0.140996,0.171981,0.220037,0.793396,...,0.145816,-0.072648,0.402713,-0.090362,-0.330037,0.479445,0.326108,0.194447,0.829705,-0.270428
107,-0.165711,-0.261316,-0.412056,0.026257,0.021744,0.003926,0.142521,0.173544,0.223019,0.799004,...,0.120646,-0.103033,0.151844,-0.006096,-0.245130,0.480137,0.322715,0.197147,0.312113,-0.062751
108,0.032262,0.165552,0.092030,0.026230,0.023923,0.003721,0.142519,0.174529,0.222930,0.795025,...,0.121020,-0.116731,0.000000,0.020000,0.087141,0.480118,0.323807,0.196075,0.000000,0.020000


In [ ]:

collist = ['wgt_cal_'+m for m in ['ew', 'est_all', 'est_cov', 'est_sds']]
df.describe()
sr_df

avg_wgt_cal, min_wgt_cal, mean_raver_port_ret, std_raver_port_ret


In [42]:

sim_results = pd.DataFrame(dtype=float, columns=['true', 'ew', 'est_all', 'est_cov', 'est_sds'], index=range(num_sims))
for s in range(num_sims):
    # print('Simulation number: ' + str(s))
    sim_results.iloc[s] = simulation(means, cov, short_lb=None, seed=s).T


In [44]:

def figplot(xvar, yvar):
    label_dict = {'true': 'theoretical optimal weights', 
                'ew': 'equal weights',
                'est_all': 'estimate all inputs',
                'est_cov': 'estimate covariance matrix only',
                'est_sds': 'estimate standard deviations only'}


    # Plot simulated Sharpe ratios
    string ="Sharpe Ratios:<br>"
    string += "  "+ label_dict[xvar] + ": %{x:0.3}<br>"
    string += "  "+ label_dict[yvar] + ": %{y:0.3}<br>"
    string += "<extra></extra>"

    trace = go.Scatter(x=sim_results[xvar],y=sim_results[yvar],mode='markers', name='A simulated outcome', hovertemplate=string)
    max_sr = sim_results.max().max()

    # Plot 45 degree line
    frac_x_beats_y = (sim_results[xvar] > sim_results[yvar]).mean()

    string ="Below this line: " + f'({frac_x_beats_y:.0%} of simulations)<br>' + ""
    string +="   `"+label_dict[xvar] + "` outperformed `" +label_dict[yvar] + "` " + "<br>"
    string +="Above this line: " + f'({1-frac_x_beats_y:.0%} of simulations)<br>' + "" 
    string +="   `"+label_dict[yvar] + "` outperformed `" +label_dict[xvar] + "`<br>"    
    string += "<extra></extra>"   
    trace_45 = go.Scatter(x=np.linspace(0,max_sr,100),y=np.linspace(0,max_sr,100),mode='lines', name='45-degree line', hovertemplate=string)
    fig = go.Figure()
    fig.add_trace(trace)
    fig.add_trace(trace_45)
    fig.layout.xaxis["title"] = "SR: " + label_dict[xvar]
    fig.layout.yaxis["title"] = "SR: " + label_dict[yvar]
    fig.update_yaxes(tickformat=".2")
    fig.update_xaxes(tickformat=".2")
    fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
    fig.update_xaxes(range=[0, 1.05 * max_sr])
    fig.update_yaxes(range=[0, 1.05 * max_sr])
    fig.show()
    


In [46]:
def histplot(xvar, yvar):
    label_dict = {'true': 'theoretical optimal weights', 
                'ew': 'equal weights',
                'est_all': 'estimate all inputs',
                'est_cov': 'estimate covariance matrix only',
                'est_sds': 'estimate standard deviations only'}

    # Plot differences in Sharpe ratios
    avg_diff = (sim_results[xvar] - sim_results[yvar]).mean()
    frac_x_beats_y = (sim_results[xvar] > sim_results[yvar]).mean()
    string = "This bin contains %{y:0.3}% of simulations <br>"
    string +="Average difference in SRs across all simulations: " + f'{avg_diff:.3f} <br>'
    string +="'" + label_dict[xvar] + "' outperforms '" + label_dict[yvar] + "' in " +  f'{frac_x_beats_y:.0%} of all simulations<br>' + "" 
    string += "<extra></extra>"   
    trace=go.Histogram(x=(sim_results[xvar] - sim_results[yvar]), hovertemplate = string )

    max_sr = sim_results.max().max()

    fig = go.Figure()
    fig.add_trace(trace)
    fig.layout.xaxis["title"] = "Difference in SRs: '" + label_dict[xvar] + "' minus '" + label_dict[yvar] + "'"
    fig.layout.yaxis["title"] = "Percent of Simulations"
    fig.update_yaxes(tickformat=".2")
    fig.update_layout(legend=dict(yanchor="top", y =0.99, xanchor="left", x=0.01))
    fig.update_xaxes(range=[-0.6*max_sr, 0.6*max_sr])
    # fig.update_yaxes(range=[0, 1.05 * sim_results.max().max()])
    fig.update_traces(marker_line_width=1,marker_line_color="black", histnorm='percent')    
    fig.show()
                  


In [47]:
figplot('ew','est_all')
figplot('ew','est_cov')
figplot('ew','est_sds')

In [48]:
histplot('ew','est_all')
histplot('ew','est_cov')
histplot('ew','est_sds')

In [43]:
sim_results_save = sim_results.copy()

### Keep risk-aversion constant each period and choose portfolio based on that risk aversion

In [14]:

seed = 10
short_lb = -3.0

# Simulate and estimate rolling inputs
rets = mvn.rvs(means, cov, size=60+T, random_state = seed)
df = pd.DataFrame(data=rets, columns=['r0','r1','r2'])
df.columns
df['mn0']=df['r0'].rolling(60).mean()
df['mn1']=df['r1'].rolling(60).mean()
df['mn2']=df['r2'].rolling(60).mean()
df['sd0']=df['r0'].rolling(60).std()
df['sd1']=df['r1'].rolling(60).std()
df['sd2']=df['r2'].rolling(60).std()

corrs = df[['r0','r1','r2']].rolling(60, min_periods=60).corr()
df['c01']=corrs.loc[(slice(None),'r0'),'r1'].values
df['c02']=corrs.loc[(slice(None),'r0'),'r2'].values
df['c12']=corrs.loc[(slice(None),'r1'),'r2'].values



In [34]:
model_list = ['true', 'ew', 'est_all', 'est_cov', 'est_sds']
for model in model_list:
    df['portret_'+model] = np.nan  # portret is the realized portfolio return of the 100% risky asset portfolio
    if model not in ['true','ew']:
        df['wgt0_'+model] = np.nan
        df['wgt1_'+model] = np.nan
        df['wgt2_'+model] = np.nan
    df['wgt_cal_'+model] =np.nan
    df['raver_portret_'+model] =np.nan #raver_portret_ is the realized return of the CAL choice of the raver investor

for i in np.arange(60,60+T):
    # Full estimation inputs at each point in time
    means = df[['mn0','mn1','mn2']].iloc[i-1].values
    sds   = df[['sd0','sd1','sd2']].iloc[i-1].values
    corr01 = df.loc[i-1,'c01']
    corr02 = df.loc[i-1,'c02']
    corr12 = df.loc[i-1,'c12']
    C  = np.identity(3)
    C[0, 1] = C[1, 0] = corr01
    C[0, 2] = C[2, 0] = corr02
    C[1, 2] = C[2, 1] = corr12
    cov = np.diag(sds) @ C @ np.diag(sds)

    ##### Note: all portfolio weights considered to be beginning of period weights
    ##### (so multiply by contemporaneous realized returns)
    # Theoretical optimal weights
    model = 'true'
    df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_true
    df.loc[i,'raver_portret_'+model] = r + cal_wgt_true * (df.loc[i,'portret_'+model] -r)

    # Full estimation tangency portfolio
    model = 'est_all'
    w0, w1, w2 = tangency(means,cov,r,short_lb)
    df.loc[i,'wgt0_' + model] = w0
    df.loc[i,'wgt1_' + model] = w1
    df.loc[i,'wgt2_' + model] = w2
    # df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
    wgts = np.array([w0, w1, w2])
    df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (wgts @ means - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)

    # Estimate only covariance matrix
    model = 'est_cov'
    w0, w1, w2 = gmv(cov,short_lb)
    df.loc[i,'wgt0_' + model] = w0
    df.loc[i,'wgt1_' + model] = w1
    df.loc[i,'wgt2_' + model] = w2
    # df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
    wgts = np.array([w0, w1, w2])
    df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)


    # Estimate only standard deviations in covariance matrix
    model = 'est_sds'
    cov[0, 1] = cov[1, 0] = 0.0
    cov[0, 2] = cov[2, 0] = 0.0
    cov[1, 2] = cov[2, 1] = 0.0
    w0, w1, w2 = gmv(cov,short_lb)
    df.loc[i,'wgt0_' + model] = w0
    df.loc[i,'wgt1_' + model] = w1
    df.loc[i,'wgt2_' + model] = w2
    # df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ df.loc[i,['wgt0_'+model,'wgt1_'+model,'wgt2_'+model]].values
    wgts = np.array([w0, w1, w2])
    df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)    

    # Equal-weighted portfolio
    model = 'ew'
    cov[0, 0] = cov[1, 1] = cov[2, 2] = sds.mean()
    wgts = (1/n)*np.ones(n)
    # df.loc[i,'portret_'+model]= df.loc[i,['r0','r1','r2']].values @ wgts_ew
    df.loc[i,'portret_'+model] = df.loc[i,['r0','r1','r2']].values @ wgts
    df.loc[i,'wgt_cal_'+model] = (means.mean() - r) / (raver * (wgts @ cov @ wgts))
    df.loc[i,'raver_portret_'+model] = r + df.loc[i,'wgt_cal_'+model]  * (df.loc[i,'portret_'+model] -r)   



portret_list = ['raver_portret_' +  model for model in model_list]
stats = df[portret_list].describe()

sr_df = pd.DataFrame(dtype=float, columns = ['sr'], index = model_list)
for model in model_list:
    sr_df.loc[model,'sr'] = (stats.loc['mean','raver_portret_' +  model] - r)/stats.loc['std','raver_portret_' +  model]

In [36]:
collist = ['wgt_cal_'+m for m in ['ew', 'est_all', 'est_cov', 'est_sds']]
df[collist].describe()
sr_df

,sr
true,0.156056
ew,0.124838
est_all,0.030235
est_cov,0.129685
est_sds,0.127453
